Збір інформації та робота з базою даних

Потрібно виконати такі кроки:  
* знайти джерело інформації (сайт, api чи якась неструктурована база даних), будь-яке джерело, відмінне від тих, що ми вже розглядали на заняттях і домашніх
* зібрати дані із обраного джерела
* спроектувати базу даних для збереження зібраних даних
* створити цю базу даних за допомогою бібліотеки sqlite3
* записати зібрану інформацію в базу даних
* одним чи кількома запитами показати вміст заповненої бази даних

Завантажте, будь ласка:  
* ноутбук з усіма кроками (файл з розширенням .ipynb)
* файл бази даних (зазвичай з розширенням .db, .sqlite, .sqlite3)

# В якості джерела буде використаний сайт https://go.drugbank.com/

збирати буду вибіркові дані щодо лікарських засобів, дані вибрані з огляду того, що вони є фізико-хімічними або біологічними властивостями кожного лікарського засобу або ж їх класифікацією, для подальшої побудови моделі, яка зможе прогнозувати дані властивості за хімічної будови потенційного лікарського засобу абож довільної молекули.

У цієї бази даних є API, проте отримати доступ до неї в мене не вдалося (складно пройти реєстрацію для одержання доступу)

# Step 1 Import libraries:

In [10]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from IPython.display import display, HTML
import sqlite3
import pandas as pd
import re
import sys

# Step 2 Make a request with requests module via a URL(Uniform Resource Locator)

In [7]:
def make_req(url:str, timeout=50)-> object:
    """Return new Request object using the library Request() from URL
    Argument:
    url : is a unique identifier used to locate a resource on the Internet 
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0',}
    resp = requests.get(url, headers=headers, timeout=timeout)
    return resp

In [8]:
url = f'https://go.drugbank.com/drugs/DB00278' # test url
resp = make_req(url)

# Step 3 Retrieve the HTML content as text.

In [9]:
resp.text

'<!DOCTYPE html><html><head><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport" /><meta content="text/html; charset=UTF-8" http-equiv="Content-Type" /><meta content="fdhmwDBTGwNPe0lKcJVly7AToLfZYvJ-mtjFAPTMZGs" name="google-site-verification" /><meta content="296E0FF6379AE64A0D18C39BB40ECD6E" name="msvalidate.01" /><meta content="DB00278" name="dc.identifier" /><meta content="Argatroban" name="dc.title" /><meta content="Argatroban is a synthetic direct thrombin inhibitor used for the prevention and treatment of thrombosis related to heparin use." name="description" /><link href="https://go.drugbank.com/drugs/DB00278" rel="canonical" /><title>Argatroban: Uses, Interactions, Mechanism of Action | DrugBank Online</title><link rel="apple-touch-icon" type="image/x-icon" href="/favicons/apple-touch-icon-57x57-precomposed.png" sizes="57x57" /><link rel="apple-touch-icon" type="image/x-icon" href="/favicons/apple-touch-icon-60x60-precomposed.png" sizes="60x60"

# Step 4 Examine the HTML structure closely to identify the particular HTML element from which to extract data.

Found that the key tags are `div` with class `drug-card` and `card-content` wich contain tags `dl`,`dt` and `dd`. And also `id` with values: 'generic-name', 'drugbank-accession-number', 'type', 'weight', 'chemical-taxonomy',
         'toxicity', 'atc-codes', 'drug-categories', 'cas-number', 'iupac-name', 'smiles','msds', 
        'experimental-properties', 'associated-conditions', 'absorption', 'volume-of-distribution', 
        'half-life',

# Step 5 Use BeautifulSoup to find the particular element from the response and extract the text.

In [6]:
# get list all id into test url
soup = BeautifulSoup(resp.text, 'html.parser')
list_id = []
for dt in soup.find_all('dt'):
    list_id.append(dt.get('id'))
print(list_id)

['summary', 'generic-name', 'drugbank-accession-number', 'background', 'type', 'groups', 'structure', 'weight', 'chemical-formula', 'synonyms', 'indication', 'associated-conditions', 'contraindications-blackbox-warnings', 'pharmacodynamics', 'mechanism-of-action', 'absorption', 'volume-of-distribution', 'protein-binding', 'metabolism', 'route-of-elimination', 'half-life', 'clearance', 'adverse-effects', 'toxicity', 'pathways', 'pharmacogenomic-effects-adrs', 'drug-interactions', 'food-interactions', None, 'product-ingredients', 'brand-name-prescription-products', 'generic-prescription-products', 'atc-codes', 'drug-categories', 'chemical-taxonomy', 'description', 'kingdom', 'super-class', 'class', 'sub-class', 'direct-parent', 'alternative-parents', 'substituents', 'molecular-framework', 'external-descriptors', 'affected-organisms', 'unii', 'cas-number', 'inchi-key', 'inchi', 'iupac-name', 'smiles', 'synthesis-reference', 'general-references', 'external-links', 'human-metabolome-databas

# Step 6 Data Extraction and Cleaning

create class wich takes object BeautifulSoup and return dictionary look like JSON

In [11]:
class ScrapingDragbank():
    """the class object takes the bs4 object as input 
    and returns a dictionary with data from go.drugbank.com"""
    
    def __init__(self, soup: object, _id:str)->None:
        self.soup = soup
        self.id = _id
        self.base_sel = self.get_base_sel()
        self.drug = self.ForeignKey()
        print(self.drug)

    def get_base_sel(self)->str:
        """Return a key tag that contains the required data"""
        return 'div.drug-card div.card-content dl'

    def get_simple_data(self, _id:str)->dict:
        """Return dictionary wich contain data with some id"""
        for i in self.soup.select(f'{self.base_sel}'):
            for dt in i.find_all_next('dt'):
                if dt.get('id') == _id:
                    return {dt.text : dt.find_next('dd').text}

    def get_href_data(self, _id:str)->dict:
        """Get href from MSDS tag"""
        for i in self.soup.select(f'{self.base_sel}'):
            for dt in i.find_all_next('dt'):
                if dt.get('id') == _id:
                    return {dt.text : dt.find_next('a').get('href')}  

    def ForeignKey(self)->str:
        """Return accession number into database drugbank """
        if self.get_simple_data('drugbank-accession-number') == None:
            return self.id
        else:
            return self.get_simple_data('drugbank-accession-number').get('DrugBank Accession Number')

    def get_experimental_properties(self)->dict:
        """Return dictionary with some physicochemical properties of drug"""
        properties = {}
        for i in self.soup.select(f'{self.base_sel}'):
            for dt in i.find_all_next('dt'):
                if dt.get('id') == 'experimental-properties':
                    table = dt.find_next('table')
                    if table != None:
                        headers = [header.text for header in table.find_all('th')]
                        res = [{headers[i]: cell.text for i, cell in enumerate(row.find_all('td'))} for row in table.find_all('tr')]
                        for i in res:
                            properties[i.get('Property')] = i.get('Value')
        if not properties:
            return {'id':self.drug}
        else:
            properties.pop(None)
            properties.update({'id':self.drug})
            return properties

    def det_drug_categories(self)-> dict:
        """Return dictionary with data categories of drug"""
        for i in self.soup.select(f'{self.base_sel}'):
            for dt in i.find_all_next('dt'):
                if dt.get('id') == 'drug-categories':
                    ulli = set()
                    for ul in dt.find_next('ul'):
                        ulli.add(ul.find_next('li').text)
                    return {self.drug :{dt.text : ulli}}

    def get_atc(self)->dict:
        """return dictionary with data Anatomical Therapeutic Chemical Classification System from database drugbank"""
        atc = {}
        for i in self.soup.select(f'{self.base_sel}'):
            for dt in i.find_all_next('dt'):
                if dt.get('id') == 'atc-codes':
                    for li in dt.find_next('ul', {'class':'atc-drug-tree'}):
                        atc.update({li.text.split('—')[0].replace(' ', ''):1})
        atc.update({'id':self.drug})
        return atc
    
    @staticmethod
    def remove_character_str(s:str)->str:
        """Return the corrected taxonomy value"""
        if s[-1] == 's':
            return s[:len(s)-1]
        else:
            return s
    
    def get_chemical_taxonomy(self)->object:
        """
        Return generator with dictionary with classifying compounds by their functional groups:
        group key in dict - is Names functional groups
        link  - has link to is a web-based application for automated structural classification of chemical entities.
        drug  - is accession number into database drugbank
        """
        for i in self.soup.select(f'{self.base_sel}'):
            for dt in i.find_all_next('dt'):
                if dt.get('id') == 'chemical-taxonomy':
                    for span in dt.find_all_next('span', {'class':'separated-list-item'}):
                        href = span.find_next('a', {'class':'classyfire-taxnode'})
                        yield {'id':self.remove_character_str(span.text), self.drug:1}
                        if href != None:
                            group = self.remove_character_str(href.text)
                            link = href.get('href')
                            description = href.get('data-content')
                            yield {'id':group, self.drug:link[-8:-1]}
        

    def get_simple_table(self)->dict:
        """Return dictionary with fields: 
                    'generic-name' - is a medication with the exact same active ingredient as 
                                     the brand-name drug, is taken the same way and offers 
                                     the same effect, 
                    'type'         - about a molecular size some drugs, 
                    'cas-number'   - is a unique identification number assigned by 
                                     the Chemical Abstracts Service, 
                    'iupac-name'   - the most 'official' rules for names of chemical compounds 
                                     are promulgated by the International Union of Pure and Applied 
                                     Chemistry (IUPAC),
                    'smiles'       - is a line notation for describing chemical structures using 
                                     short ASCII strings, 
                    'msds'         - link to a Material Safety Data Sheet (MSDS) is a document 
                                     that contains information on the potential hazards
                    """
        general_data = {}
        for _id in ['generic-name', 'type', 'cas-number', 'iupac-name',
                   'smiles', 'msds']:
            if self.get_simple_data(_id) == None:
                continue
            else:
                general_data.update(self.get_simple_data(_id))
            if self.get_href_data('msds') == None:
                continue
            else:
                general_data.update(self.get_href_data('msds'))
        general_data.update({'id':self.drug})
        return general_data

    def get_weight(self)->dict:
        """Return dictionary with weight values of drugs"""
        weight = {}
        weight.update({'id':self.drug})
        if self.get_simple_data('weight') == None:
            return weight
        else:
            s = self.get_simple_data('weight')['Weight']
            for j, i in enumerate(s.split(' ')):
                if i.find(':') == -1:
                    continue
                else:
                    weight.update({'Weight_' + s.split(' ')[j].replace(':', '') : float(s.split(' ')[j+1])})
            return weight

# Step 7 Design a database and recording collected data

In [12]:
class FillDataBase():
    """Class wich recording collected data"""
    def __init__(self, Dict:dict, table:str, DB:object)->None:
        """Take arguments for writing data into database
        Arguments:
                    Dict : dictionary with some data, mast have key with id
                    table : name of table in database
                    DB : object database sqlite3
        """
        self.Dict = self.new_dict(Dict)
        self.table_name_DB = table
        self.DB = DB
        self.columns = self.get_columns()
        self.placeholder = self.get_placeholders()
        self.con = sqlite3.connect(self.DB)

        
    @staticmethod
    def clean_name(s:str)-> str:
        """Return new name of field wich filtered from unwanted characters"""
        reg = re.compile('[^a-zA-Z0-9—_]')
        s = s.replace('-','—').replace(' ', '_')
        return reg.sub('', s)
    
    def new_dict(self, Dict:dict)->dict:
        """Return dictionary with filered keys"""
        return {self.clean_name(k):v for k,v in Dict.items()}

    def get_columns(self)->str:
        """Return columns string for SQL query"""
        return str(list(self.Dict.keys())).replace('[', '').replace(']','').replace("'", '')
    
    def get_placeholders(self)->str:
        """Return columns string for SQL query"""
        return ':'+', :'.join(self.Dict.keys())

    def CREATE_TABLE(self)->None:
        """Create table into database"""
        cursor = self.con.cursor()
        query = f"CREATE TABLE IF NOT EXISTS {self.table_name_DB} (id VARCHAR(17) PRIMARY KEY);"
        cursor.execute(query)

    def ALTER_TABLE(self, column:str)-> None:
        """Add some columns into table of database"""
        cursor = self.con.cursor()
        cursor.execute('ALTER TABLE "%s" ADD "%s" ;' % (self.table_name_DB, column))

    def UPDATE(self, value_id:[str,int,float], column_value:str, column:str)->None:
        """"
        Update values into some table of database
        Arguments:
                    value_id: new value wich mast update
                    column_value: name of column of table
                    column: name column wich mast have replase data
        See: https://pynative.com/python-sqlite-update-table/"""
        try:
            sqliteConnection = sqlite3.connect(self.DB)
            cursor = sqliteConnection.cursor()
            # print("Connected to SQLite")

            sql_update_query = f"""UPDATE {self.table_name_DB} SET {column} = ? where id = ?"""
            #sql_update_query = f"""UPDATEUpdate {taxonomy} set {column} = ? where id = ?"""
            data = (column_value, value_id)
            cursor.execute(sql_update_query, data)
            sqliteConnection.commit()
            # print("Record Updated successfully")
            cursor.close()

        except sqlite3.Error as error:
            print("Failed to update sqlite table", error)
        finally:
            if sqliteConnection:
                sqliteConnection.close()
                # print("The sqlite connection is closed")

    def INSERT_INTO(self)-> None:
        """Insert data and columns into table of database"""
        cursor = self.con.cursor()
        query = 'INSERT OR IGNORE INTO %s (%s) VALUES (%s);' % (self.table_name_DB, self.columns, self.placeholder)
        # query = 'INSERT INTO %s (%s) VALUES (%s);' % (self.table_name_DB, self.columns, self.placeholder)
        # https://stackoverflow.com/questions/36518628/sqlite3-integrityerror-unique-constraint-failed-when-inserting-a-value
        try:
            cursor.execute(query, self.Dict)
            self.con.commit()
        except sqlite3.Error as e:
            if e.args[0][0:26+len(self.table_name_DB)] == f'table {self.table_name_DB} has no column named':
                for column in self.Dict.keys():
                    try:
                        self.ALTER_TABLE(column)
                    except sqlite3.Error as E:
                        if E.args[0][0:22] == f'duplicate column name:':
                            continue
                        else:
                            print('Some ERROR FROM  def INSERT_INTO')
                            print(e.args)
                            print(E.args)
                            sys.exit()
            elif e.args[0][:24] == 'UNIQUE constraint failed':
                pass
            else:
                print(e.args, 'FROM def INSERT_INTO')
            self.INSERT_INTO()

# Step 8 Scraping Script

In [13]:
DB = 'FromDrugBankData.db'

for _key in range(1,10):
    DB_key = str(_key).zfill(5)
    url = f'https://go.drugbank.com/drugs/DB{DB_key}'
    resp = make_req(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    obj = ScrapingDragbank(soup, str('DB' + DB_key))
    table_DATA = {
    'atc': obj.get_atc(),
    'describe': obj.get_simple_table(),
    'weight': obj.get_weight(),
    'properties': obj.get_experimental_properties(),
    # 'taxonomy' : obj.get_chemical_taxonomy()
        }
    for table in table_DATA.keys():
        objF = FillDataBase(table_DATA[table], table, DB)
        objF.CREATE_TABLE()
        objF.INSERT_INTO()

    FK = obj.ForeignKey()
    taxonomy = obj.get_chemical_taxonomy()
    for d in taxonomy:
        objF = FillDataBase(d, 'taxonomy', DB)
        objF.CREATE_TABLE()
        objF.INSERT_INTO()
        for KEY in d:
            value_id = d['id']
            if KEY != 'id':
                objF.UPDATE(value_id, d[FK], FK)

DB00001
DB00002
DB00003
DB00004
DB00005
DB00006
DB00007
DB00008
DB00009


## Step 10 Viewing Information About Database

In [21]:
def Query(SQL_query, DB, return_=False):
    try:
        with sqlite3.connect(DB) as con: # Create DB or connection
            df = pd.read_sql_query(f"{SQL_query}", con)
            if return_ == True:
                return df
            else:
                display(HTML(df.to_html()))
    except sqlite3.Error as e:
        print(e)

In [22]:
tables_list = Query(f'SELECT * FROM sqlite_schema', DB, return_=1)['tbl_name'].unique()
for table in tables_list:
    Query(f'SELECT * FROM {table}', DB)

,id,B01AE,B01A,B01,B,L01FE,L01F,L01,L,R05CB,R05C,R05,R,L01XX,L01X,L04AB,L04A,L04,L02AE,L02A,L02,L03AB,L03A,L03,B01AD
0,DB00001,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DB00002,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DB00003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DB00004,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DB00005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DB00006,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DB00007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
7,DB00008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN
8,DB00009,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


,id,Generic_Name,Type,CAS_number,MSDS,IUPAC_Name,SMILES
0,DB00001,Lepirudin,Biotech,138068-37-8,None,None,None
1,DB00002,Cetuximab,Biotech,205923-56-4,//s3-us-west-2.amazonaws.com/drugbank/msds/DB00002.pdf?1458412933,None,None
2,DB00003,Dornase alfa,Biotech,143831-71-4,//s3-us-west-2.amazonaws.com/drugbank/msds/DB00003.pdf?1410714694,None,None
3,DB00004,Denileukin diftitox,Biotech,173146-27-5,None,None,None
4,DB00005,Etanercept,Biotech,185243-69-0,None,None,None
5,DB00006,Bivalirudin,Small Molecule,128270-60-0,None,"(4S)-4-[(2S)-2-[(2S)-2-[(2S)-2-{2-[(2S)-2-(2-{2-[2-(2-{[(2S)-1-[(2S)-2-{[(2S)-1-[(2R)-2-amino-3-phenylpropanoyl]pyrrolidin-2-yl]formamido}-5-carbamimidamidopentanoyl]pyrrolidin-2-yl]formamido}acetamido)acetamido]acetamido}acetamido)-3-carbamoylpropanamido]acetamido}-3-carboxypropanamido]-3-phenylpropanamido]-4-carboxybutanamido]-4-{[(2S,3S)-1-[(2S)-2-{[(1S)-3-carboxy-1-{[(1S)-3-carboxy-1-{[(1S)-1-{[(1S)-1-carboxy-3-methylbutyl]carbamoyl}-2-(4-hydroxyphenyl)ethyl]carbamoyl}propyl]carbamoyl}propyl]carbamoyl}pyrrolidin-1-yl]-3-methyl-1-oxopentan-2-yl]carbamoyl}butanoic acid",CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O
6,DB00007,Leuprolide,Small Molecule,53714-56-0,None,(2S)-1-[(2S)-5-carbamimidamido-2-[(2S)-2-[(2R)-2-[(2S)-2-[(2S)-3-hydroxy-2-[(2S)-2-[(2S)-3-(1H-imidazol-4-yl)-2-{[(2S)-5-oxopyrrolidin-2-yl]formamido}propanamido]-3-(1H-indol-3-yl)propanamido]propanamido]-3-(4-hydroxyphenyl)propanamido]-4-methylpentanamido]-4-methylpentanamido]pentanoyl]-N-ethylpyrrolidine-2-carboxamide,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](CC(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=C1C=CC=C2)NC(=O)[C@H](CC1=CNC=N1)NC(=O)[C@@H]1CCC(=O)N1
7,DB00008,Peginterferon alfa-2a,Biotech,198153-51-4,None,None,None
8,DB00009,Alteplase,Biotech,105857-23-6,None,None,None


,id,Weight_Average,Weight_Monoisotopic
0,DB00001,NaN,NaN
1,DB00002,NaN,NaN
2,DB00003,NaN,NaN
3,DB00004,NaN,NaN
4,DB00005,NaN,NaN
5,DB00006,2180.2853,2178.985813
6,DB00007,1209.3983,1208.645462
7,DB00008,NaN,NaN
8,DB00009,NaN,NaN


,id,melting_point_C,water_solubility,isoelectric_point,hydrophobicity,pKa
0,DB00001,65 °C,Soluble,3.7,None,None
1,DB00002,"61 °C (FAB fragment), 71 °C (whole mAb)",None,8.48,-0.413,None
2,DB00003,67 °C,None,4.58,-0.083,None
3,DB00004,None,None,5.45,-0.301,None
4,DB00005,71 °C (whole mAb),None,7.89,-0.529,None
5,DB00006,None,None,None,None,None
6,DB00007,150-155,None,None,None,9.6
7,DB00008,61 °C,None,5.99,None,None
8,DB00009,60 °C,None,7.61,-0.516,None


,id,DB00006
0,Hexacarboxylic acids and derivative,C000296
1,Peptide,C000034
2,Tyrosine and derivative,C000431
3,Phenylalanine and derivative,C000432
4,Glutamic acid and derivative,C000432
5,Asparagine and derivative,C000431
6,Aspartic acid and derivative,C000431
7,Isoleucine and derivative,C000433
8,Leucine and derivative,C000432
9,Proline and derivative,C000432
